# 正向传播、反向传播和计算图
前⾯⼏节⾥我们使⽤了小批量随机梯度下降的优化算法来训练模型。在实现中，我们只提供了模
型的正向传播（forward propagation）的计算，即对输⼊计算模型输出，然后通过autograd模块
来调⽤系统⾃动⽣成的backward函数计算梯度。基于反向传播（back-propagation）算法的⾃动
求梯度极⼤简化了深度学习模型训练算法的实现。本节我们将使⽤数学和计算图（computational
graph）两个⽅式来描述正向传播和反向传播。具体来说，我们将以带L2范数正则化的含单隐藏
层的多层感知机为样例模型解释正向传播和反向传播
## 正向传播
正向传播是指对神经⽹络沿着从输⼊层到输出层的顺序，依次计算并存储模型的中间变量（包括
输出）。为简单起⻅，假设输⼊是⼀个特征为$\boldsymbol{x} \in \boldsymbol{\mathbb R}^d$的样本，且不考虑偏差项，那么中间变量
$z = W^{(1)}x$

其中$W^{(1)} \in \mathbb R^{h×d}$是隐藏层的权重参数。把中间变量$z \in \mathbb R^h$输⼊按元素运算的激活函数$\phi$后，将
得到向量⻓度为$h$的隐藏层变量$h=\phi(z)$

隐藏层变量$h$也是⼀个中间变量。假设输出层参数只有权重$W^{(2)} \in \mathbb R^{q×h}$，可以得到向量⻓度
为$q$的输出层变量$o=W^{(2)}h$
假设损失函数为$\ell$，且样本标签为$y$，可以计算出单个数据样本的损失项
$L=\ell(o,y)$

根据$L_2$范数正则化的定义，给定超参数λ，正则化项即$$s = \frac{\lambda}{2} \left(\|\boldsymbol{W}^{(1)}\|_F^2 + \|\boldsymbol{W}^{(2)}\|_F^2\right),$$
其中矩阵的Frobenius范数等价于将矩阵变平为向量后计算$L_2$范数。
矩阵A的Frobenius范数定义为矩阵A各项元素的绝对值平方的总和，即

$$\|A\|_F=\sqrt{\sum_i \sum_j X_{i,j}^2}$$

最终，模型在给定的数据样本上带正则化的损失为$J=L+s$

我们将J称为有关给定数据样本的⽬标函数，并在以下的讨论中简称⽬标函数
## 正向传播的计算图
我们通常绘制计算图来可视化运算符和变量在计算中的依赖关系。图3.6绘制了本节中样例模型
正向传播的计算图，其中左下⻆是输⼊，右上⻆是输出。可以看到，图中箭头⽅向⼤多是向右和
向上，其中⽅框代表变量，圆圈代表运算符，箭头表⽰从输⼊到输出之间的依赖关系。

![正向传播的计算图](../img/forward.svg)
## 反向传播
反向传播指的是计算神经⽹络参数梯度的⽅法。总的来说，反向传播依据微积分中的链式法则，
沿着从输出层到输⼊层的顺序，依次计算并存储⽬标函数有关神经⽹络各层的中间变量以及参数
的梯度。对输⼊或输出$X,Y,Z$为任意形状张量的函数$Y = f(X)和Z = g(Y)$，通过链式法则，我们有

$$\frac {\partial Z} {\partial X} = \text{prod}\left(\frac {\partial Z} {\partial Y},\frac {\partial Y} {\partial X}\right)$$

其中prod运算符将根据两个输⼊的形状，在必要的操作（如转置和互换输⼊位置）后对两个输⼊做乘法

回顾⼀下本节中样例模型，它的参数是$W^{(1)}和W^{(2)}$， 因 此 反 向 传 播 的 ⽬ 标 是 计
算$\partial J/\partial W^{(1)}和\partial J/\partial W^{(2)}$。我们将应⽤链式法则依次计算各中间变量和参数的梯度，其计
算次序与前向传播中相应中间变量的计算次序恰恰相反。⾸先，分别计算⽬标函数$J = L + s$有
关损失项L和正则项s的梯度

$$\frac {\partial J} {\partial L} = 1, \frac {\partial J} {\partial s} = 1$$

其次，依据链式法则计算⽬标函数有关输出层变量的梯度$\partial J/\partial o \in \mathbb{R}^q$

$$\frac {\partial J} {\partial o} = \text{prod}\left(\frac {\partial J} {\partial L},\frac {\partial L} {\partial o}\right) = \frac {\partial L} {\partial o}$$

接下来，计算正则项有关两个参数的梯度：

$$\frac {\partial s} {\partial W^{(1)}} = \lambda W^{(1)}, \frac {\partial s} {\partial W^{(2)}} = \lambda W^{(2)}$$

现在，我们可以计算最靠近输出层的模型参数的梯度$\partial J/\partial W^{(2)} \in \mathbb{R}^{q\times h}$。依据链式法则，得到

$
\frac{\partial J}{\partial \boldsymbol{W}^{(2)}}
    = \text{prod}\left(\frac{\partial J}{\partial \boldsymbol{o}}, \frac{\partial \boldsymbol{o}}{\partial \boldsymbol{W}^{(2)}}\right) + \text{prod}\left(\frac{\partial J}{\partial s}, \frac{\partial s}{\partial \boldsymbol{W}^{(2)}}\right)
    = \frac{\partial J}{\partial \boldsymbol{o}} \boldsymbol{h}^\top + \lambda \boldsymbol{W}^{(2)}.
$

沿着输出层向隐藏层继续反向传播，隐藏层变量的梯度$\partial J/\partial h \in \mathbb{R}^h$可以这样计算：

$
\frac{\partial J} {\partial h} = \text{prod}\left(\frac {\partial J} {\partial o},\frac {\partial o} {\partial h}\right) 
    = {W^{(2)}}^\top \frac{\partial J} {\partial o}
$

由于激活函数$\phi$是按元素运算的，中间变量z的梯度$\partial J/\partial z \in \mathbb{R}^h$的计算需要使⽤按元素乘法符$\odot$：

$
\frac{\partial J} {\partial z} = \text{prod}\left(\frac {\partial J} {\partial h},\frac {\partial h} {\partial z}\right) 
    = \frac {\partial J} {\partial h} \odot \phi'(z)
$

最终，我们可以得到最靠近输⼊层的模型参数的梯度$\partial J/\partial W^{(1)} \in \mathbb{R}^{h\times d}$。依据链式法则，得到

$
\frac{\partial J}{\partial \boldsymbol{W}^{(1)}}
    = \text{prod}\left(\frac{\partial J}{\partial \boldsymbol{z}}, \frac{\partial \boldsymbol{z}}{\partial \boldsymbol{W}^{(1)}}\right) + \text{prod}\left(\frac{\partial J}{\partial s}, \frac{\partial s}{\partial \boldsymbol{W}^{(1)}}\right)
    = \frac{\partial J}{\partial \boldsymbol{z}} \boldsymbol{x}^\top + \lambda \boldsymbol{W}^{(1)}.
$
## 训练深度学习模型
在训练深度学习模型时，正向传播和反向传播之间相互依赖。下⾯我们仍然以本节中的样例模型
分别阐述它们之间的依赖关系

⼀⽅⾯，正向传播的计算可能依赖于模型参数的当前值，而这些模型参数是在反向传播的梯度
计算后通过优化算法迭代的。例如，计算正则化项$$s = \frac{\lambda}{2} \left(\|\boldsymbol{W}^{(1)}\|_F^2 + \|\boldsymbol{W}^{(2)}\|_F^2\right),$$依赖模型参
数$W^{(1)}和W^{(2)}$的当前值，而这些当前值是优化算法最近⼀次根据反向传播算出梯度后迭代得到
的。

另⼀⽅⾯，反向传播的梯度计算可能依赖于各变量的当前值，而这些变量的当前值是通过正向传
播计算得到的。举例来说，参数梯度$\partial J/\partial \boldsymbol{W}^{(2)} = (\partial J / \partial \boldsymbol{o}) \boldsymbol{h}^\top + \lambda \boldsymbol{W}^{(2)}$的计算需要依赖隐藏层变
量的当前值h。这个当前值是通过从输⼊层到输出层的正向传播计算并存储得到的

因此，在模型参数初始化完成后，我们交替地进⾏正向传播和反向传播，并根据反向传播计算的
梯度迭代模型参数。既然我们在反向传播中使⽤了正向传播中计算得到的中间变量来避免重复计
算，那么这个复⽤也导致正向传播结束后不能⽴即释放中间变量内存。这也是训练要⽐预测占⽤
更多内存的⼀个重要原因。另外需要指出的是，这些中间变量的个数⼤体上与⽹络层数线性相关，
每个变量的⼤小跟批量⼤小和输⼊个数也是线性相关的，它们是导致较深的神经⽹络使⽤较⼤批
量训练时更容易超内存的主要原因。

## 小结
- 正向传播沿着从输⼊层到输出层的顺序，依次计算并存储神经⽹络的中间变量。
- 反向传播沿着从输出层到输⼊层的顺序，依次计算并存储神经⽹络中间变量和参数的梯度。
- 在训练深度学习模型时，正向传播和反向传播相互依赖。